In [1]:
# Proc Data
# app to prepare the data and send it to the picker

In [2]:
import numpy as np
import pandas as pd
import os
import json
import requests
import obspy
import matplotlib.pyplot as plt
import scipy.stats as stats

from scipy.signal import lfilter, butter, decimate, hann, find_peaks

from flask import Flask, request, jsonify

In [3]:
# Import configuration paramaters for pipeline
import pconf

In [4]:
#%matplotlib widget
%matplotlib inline

In [5]:
# App variables
app = Flask(__name__)
app_name = 'proc_data'
napp = 'picker'

In [6]:
# Filters and stuff
def DAT_normalize(X):
    X = X - np.expand_dims(np.mean(X,1),1)
    X = X / np.expand_dims(np.expand_dims(np.abs(X).max(1).max(1), 1), 1)
    return X

def butter_bandpass(lowcut, highcut, fs, order=8):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq

    b, a = butter(order, [low, high], btype='band')
    return b, a

def DAT_filter(X, pdict, order=3):
    lowcut = pdict['f_low']
    highcut = pdict['f_hig']
    fs = pdict['r_smp']
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return lfilter(b, a, X, axis=1) 

def DAT_taper(X, taper_percentage=.05):
    npts = X.shape[1]
    taper_len = int(npts * taper_percentage)
    taper_sides = hann(2 * taper_len + 1)
    taper = np.hstack((taper_sides[:taper_len], np.ones(npts - taper_len)))
    return X * np.reshape(taper,(1,-1,1))

In [7]:
def read_mseed(full_path, pdict):
    # Read the trace and return it with zeros on any gaps
    # The data is also returned resampled to the r_smp rate
    return obspy.read(full_path).merge(fill_value = 0).resample(pdict['r_smp'])

In [8]:
# Read streams
def read_streams(pdict):
    streams = []
    for f in pdict['files']:
        streams.append(read_mseed(f['data_file'], pdict))
    return streams

In [9]:
def gen_data(pdict):
    streams = read_streams(pdict)
    try:
        wnum = 0
        while True:
            batch = {}
            batch['X'] = []
            batch['metadata'] = []
            batch['window_size'] = pdict['window_size']
            batch['dt'] = pdict['dt']
            batch['c_len'] = pdict['c_len']
            batch['r_smp'] = pdict['r_smp']
            batch['cat'] = pdict['cat']
            batch['trim'] = pdict['trim']
            batch['clean_cat'] = pdict['clean_cat']
            batch['events'] = pdict['events']
            for i, stream in enumerate(streams):
                start = obspy.UTCDateTime(stream[0].stats.starttime + (wnum * (pdict['window_size']-pdict['overlap'])))
                end = obspy.UTCDateTime(start + pdict['window_size'])
                # If there isnt enough data to form another cut it short
#                 if (end > obspy.UTCDateTime(stream[0].stats.endtime)):
#                     end = obspy.UTCDateTime(stream[0].stats.endtime)
                st = stream.slice(starttime=start, endtime=end)
                # Append the three channels stack to the data array
                batch['X'].append(np.vstack((st[0].data, st[1].data, st[2].data)).T)
                metadata = {
                    'sta': pdict['files'][i]['sta'],
                    'st_lat': pdict['files'][i]['st_lat'],
                    'st_lon': pdict['files'][i]['st_lon'],
                    'start': str(start),
                }
                batch['metadata'].append(metadata)
            batch['X'] = DAT_normalize(DAT_taper(DAT_filter(batch['X'], pdict))).tolist()
            wnum += 1
            yield batch
    except Exception as e:
        print(e)
        pass

In [10]:
# Test up method
@app.route('/')
def apitest():
    return f'{app_name} is working'

In [11]:
# This is the URI that process the received requests
@app.route(f'/{app_name}', methods=['POST'])
def process_request():
    # Exract json from request (received)
    pdict = request.json
    streams = read_streams(pdict)
    for stream in streams:
        print(stream)
    gen = gen_data(pdict) 
    for i in range(pdict['n_windows']):
        b = next(gen)
        # Note that only the last response is saved and returned
        picker_response = forward(b)
    
    # Return the response of processing the last batch.
    return picker_response.json()

In [12]:
def forward(json_data):
    response = requests.post(f'http://{pconf.host}:{pconf.apps[napp]}/{napp}', json = json_data, headers = pconf.head)
    return response

In [ ]:
if __name__ == '__main__':
    app.run(pconf.host, debug=False, port=pconf.apps[app_name])

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6002/ (Press CTRL+C to quit)


3 Trace(s) in Stream:
US.BMO.00.BH1 | 2020-06-01T00:00:00.000001Z - 2020-06-01T23:59:59.950001Z | 20.0 Hz, 1728000 samples
US.BMO.00.BH2 | 2020-06-01T00:00:00.000001Z - 2020-06-01T23:59:59.950001Z | 20.0 Hz, 1728000 samples
US.BMO.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.BOZ.00.BH1 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.BOZ.00.BH2 | 2020-06-01T00:00:00.024998Z - 2020-06-01T23:59:59.974998Z | 20.0 Hz, 1728000 samples
US.BOZ.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.DUG.00.BH1 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.DUG.00.BH2 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.DUG.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.

127.0.0.1 - - [10/Nov/2021 22:59:27] "POST /proc_data HTTP/1.1" 200 -


3 Trace(s) in Stream:
US.BMO.00.BH1 | 2020-06-01T00:00:00.000001Z - 2020-06-01T23:59:59.950001Z | 20.0 Hz, 1728000 samples
US.BMO.00.BH2 | 2020-06-01T00:00:00.000001Z - 2020-06-01T23:59:59.950001Z | 20.0 Hz, 1728000 samples
US.BMO.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.BOZ.00.BH1 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.BOZ.00.BH2 | 2020-06-01T00:00:00.024998Z - 2020-06-01T23:59:59.974998Z | 20.0 Hz, 1728000 samples
US.BOZ.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.DUG.00.BH1 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.DUG.00.BH2 | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
US.DUG.00.BHZ | 2020-06-01T00:00:00.000000Z - 2020-06-01T23:59:59.950000Z | 20.0 Hz, 1728000 samples
3 Trace(s) in Stream:
US.

127.0.0.1 - - [10/Nov/2021 23:04:02] "POST /proc_data HTTP/1.1" 200 -
